## Тестирование пайплайна

In [1]:
import os

os.chdir('..')

#### Embedding модель

In [2]:
from langchain_ollama import OllamaEmbeddings

# Create Embeddings using Ollama
embeddings = OllamaEmbeddings(
    model="bge-m3",
)

#### Splitter

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            model_name="gpt-4",
            chunk_size=800,
            chunk_overlap=400,
        )

In [4]:
from langchain_community.document_loaders import PyPDFLoader

filepath = "data/user-guide.pdf"

loader = PyPDFLoader(filepath)

init_docs = loader.load()

split = recursive_splitter.split_documents(init_docs)

#### Хранение в векторной БД

In [5]:
from langchain_community.vectorstores import FAISS


vector_store = FAISS.from_documents(documents=split, embedding=embeddings)

In [6]:
retriever = vector_store.as_retriever()

#### Создание Chain

In [31]:
import ollama

system_prompt = """Ты – виртуальный помощник, работающий в режиме RAG (Retrieval-Augmented Generation), который даёт ответы строго на основе предоставленных выдержек из «Руководства пользователя».

Основные правила:

1. Отвечай только по предоставленному контексту. Если нужная информация отсутствует во фрагментах, сообщай, что у тебя нет достаточных сведений.
2. Не добавляй никаких домыслов или внешних знаний. Избегай ссылок на источники вне текущего контекста.
3. Если вопрос выходит за пределы данных, предоставленных в контексте, прямо укажи, что «в предоставленном тексте нет информации по этому вопросу».
4. Старайся формулировать ответы лаконично, ясно выделяя ключевые моменты из контекста.
5. Не передавай конфиденциальных данных и не измышляй факты.
6. Оборачивай код в тройные обратные кавычки (Например, ```bash) и указывай источник, если это необходимо.

Пример поведения

Вопрос: «как отменить погашение аварии?»
Контекст: ... (выдержки из «Руководства пользователя») ...
Ответ: «Для отмены погашения аварии выполните следующие шаги:

Нажмите кнопку на панели режимов отображения.
В выпадающем списке вверху окна выберите "Активные аварии".
Выберите погашенную аварию, которую нужно вернуть в предыдущее состояние, и нажмите на неё правой кнопкой мыши.
В открывшемся контекстном меню выберите "Отменить погашение".
После выполнения этих действий авария перейдёт в состояние, которое было до погашения.
Также отменить погашение аварии можно с помощью REST запроса "Отменить погашение аварии"
Для этого запроса требуется ID аварии.
```bash
login=<...>
password=<...>
incident_id=<...>
saymon_hostname=<...>
url=https://$saymon_hostname/node/api/incidents/$incident_id/undo-clear
curl -X POST $url -u $login:$password
```
Ссылка на источник: Руководство пользователя, страница 13.»
"""


def ollama_llm(question, context):
    formatted_prompt = f"""Вопрос: "{question}"\n\Контекст: {context}"""
    response = ollama.chat(
        model="llama3.1",
        stream=False,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": formatted_prompt},
        ],
    )
    return response["message"]["content"]

In [32]:
def format_docs(docs):
    return "\n\n".join(f"""Текст контекста: {doc.page_content}, Страница: {doc.metadata.get('page_label')}""" for doc in docs)

In [33]:
# Define RAG Chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [34]:
question = "как посмотреть списки активных аварий"

In [35]:
retriever.invoke(question)

[Document(id='e4833d91-41a2-48e7-8689-3e56972577a5', metadata={'source': 'data/user-guide.pdf', 'page': 11, 'page_label': '12'}, page_content='5\n1.3. Просмотр списка активных аварий\nДля просмотра списка активных аварий требуется выполнить следующие действия:\n1. Нажать кнопку \uf0a2 в панели режимов отображения.\n2. В выпадающем списке вверху окна выбрать "Активные аварии".\nРис. 5. Список активных аварий\nЧерез столбец Путь можно перейти к родительским объектам аварийного элемента. Если\nвключен режим мультиродителя на каком-либо из уровней иерархии, имя объекта этого уровня\nвыделяется жирным шрифтом. При щелчке по такому объекту открывается список всех\nродительских объектов со ссылками на них:\nРис. 6. Мультиродитель в списке активных аварий\n1.3.1. REST API\nПолучить список активных аварий можно с помощью запроса Получить все аварии.\nРежим мультиродителя можно задать в параметрах объекта.\nNOTE\nlogin=<...> \npassword=<...> \nsaymon_hostname=<...> \nurl=https://$saymon_hostname

In [36]:
response = rag_chain(question)

In [37]:
print(response)

Для просмотра списков активных аварий:

1. Нажмите кнопку  в панели режимов отображения.
2. В выпадающем списке вверху окна выберите "Активные аварии".

Режим мультиродителя можно задать через REST API с помощью запроса Получить все аварии.


#### Экспериментирование с Splitter

In [ ]:
filepath = "data/user-guide.pdf"

In [ ]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

loader = PDFMinerPDFasHTMLLoader(filepath)
docs = loader.load()
docs[0]

Document(metadata={'source': 'data/user-guide.pdf'}, page_content='<html><head>\n<meta http-equiv="Content-Type" content="text/html">\n</head><body>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:594px; height:841px;"></span>\n<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>\n<div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:179px; top:453px; width:367px; height:27px;"><span style="font-family: NotoSerif; font-size:27px">Руководство пользователя\n<br></span></div><div style="position:absolute; border: textbox 1px solid; writing-mode:lr-tb; left:385px; top:491px; width:160px; height:11px;"><span style="font-family: NotoSerif; font-size:11px">Центральный Пульт 3.16.91\n<br></span></div><span style="position:absolute; border: black 1px solid; left:0px; top:50px; width:594px; height:152319px;"></span>\n<span style="position:absolute; border: gray 1px solid; left:0px; top:941px; width:594px; height:84

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(docs[0].page_content, "html.parser")
content = soup.find_all("div")

In [ ]:
import re

cur_fs = None
cur_text = ""
snippets = []  # first collect all snippets that have the same font size
for c in content:
    sp = c.find("span")
    if not sp:
        continue
    st = sp.get("style")
    if not st:
        continue
    fs = re.findall("font-size:(\d+)px", st)
    if not fs:
        continue
    fs = int(fs[0])
    if not cur_fs:
        cur_fs = fs
    if fs == cur_fs:
        cur_text += c.text
    else:
        snippets.append((cur_text, cur_fs))
        cur_fs = fs
        cur_text = c.text
snippets.append((cur_text, cur_fs))

In [ ]:
snippets[16]

('1.2. Оставить комментарий к аварии\n', 14)

In [ ]:
len(snippets)

1236

In [ ]:
from langchain_core.documents import Document

cur_idx = -1
semantic_snippets = []
HEADER_REGEX = r'^\d+\.\d+(?:\.\d+)*\.?\s+'
# Assumption: headings have higher font size than their respective content
for s in snippets:
    # if current snippet's font size > previous section's heading => it is a new heading
    if (
        not semantic_snippets
        or (s[1] > semantic_snippets[cur_idx].metadata["heading_font"] and 10 < s[1] < 15)
    ):
        metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
        metadata.update(docs[0].metadata)
        semantic_snippets.append(Document(page_content="", metadata=metadata))
        cur_idx += 1
        continue

    # if current snippet's font size <= previous section's content => content belongs to the same section (one can also create
    # a tree like structure for sub sections if needed but that may require some more thinking and may be data specific)
    if (
        not semantic_snippets[cur_idx].metadata["content_font"]
        or s[1] <= semantic_snippets[cur_idx].metadata["content_font"]
    ):
        semantic_snippets[cur_idx].page_content += s[0]
        semantic_snippets[cur_idx].metadata["content_font"] = max(
            s[1], semantic_snippets[cur_idx].metadata["content_font"]
        )
        continue

    # if current snippet's font size > previous section's content but less than previous section's heading than also make a new
    # section (e.g. title of a PDF will have the highest font size but we don't want it to subsume all sections)
    metadata = {"heading": s[0], "content_font": 0, "heading_font": s[1]}
    metadata.update(docs[0].metadata)
    semantic_snippets.append(Document(page_content="", metadata=metadata))
    cur_idx += 1

print(semantic_snippets[4])

page_content='3.1. Настроить параметры потока
Основные действия в требуемой последовательности:
1. Нажать левой кнопкой мыши на поток.
2. В разделе "Параметры" нажать на цветовое обозначение потока и выбрать на появившейся
палитре желаемый цвет потока:
Рис. 21. Выбор цвета потока
3. В выпадающем списке "Тип связи" выбрать желаемый вариант.
Для применения изменений нажать на кнопку   Применить . Для отмены изменений нажать
на кнопку  Отмена
3.1.1. REST API
Настроить параметры потока через REST API можно с помощью запроса Обновить поток.
Для этого запроса требуется ID потока.
Пример запроса через curl
login=<...> 
password=<...> 
saymon_hostname=<...> 
flow_id=<...> 
url=https://$saymon_hostname/node/api/flows/$flow_id 
curl -X PATCH $url -u $login:$password \ 
    -H "Content-Type: application/json" \ 
    -d @- <<EOF 
{ 
    "client_data": { 
        "color": "#111111", 
        "connectorStyle": "Bezier" 
    } 
} 
EOF
27
3.2. Создать поток
Основные действия в требуемой последователь

In [ ]:
import re
from langchain_core.documents import Document

# Регулярное выражение, проверяющее «две цифры через точку», например "1.1." или "2.10.":
# - ^\d+\.\d+   означает "числа.числа" в начале строки
# - (?:\.\d+)*  означает "необязательные группы .числа" (если вдруг бывают 1.2.3)
# - \.?         означает "необязательная точка" (чтобы поймать и "1.1" и "1.1.")
# - \s+         хотя бы один пробел
HEADER_REGEX = r'^\d+\.\d+(?:\.\d+)*\.?\s+'

cur_idx = -1
semantic_snippets = []

for text_chunk, font_size in snippets:
    # Условие НОВОГО «подзаголовка» (подглавы):
    # 1) либо нет ни одного собранного Document
    # 2) либо font_size больше, чем у текущего заголовка,
    #    font_size в диапазоне (10, 15),
    #    И сам текст совпадает с HEADER_REGEX
    if (
        not semantic_snippets
        or (
            font_size > semantic_snippets[cur_idx].metadata["heading_font"]
            and 10 < font_size < 15
            and re.match(HEADER_REGEX, text_chunk.strip())
        )
    ):
        # Начинаем новый документ
        metadata = {
            "heading": text_chunk.strip(),
            "content_font": 0,
            "heading_font": font_size,
        }
        # Передаём метаданные исходного PDF, если нужно
        metadata.update(docs[0].metadata)
        semantic_snippets.append(Document(page_content="", metadata=metadata))
        cur_idx += 1
        continue

    # Если шрифт <= content_font (или content_font == 0) текущей "главы/подглавы":
    # добавляем текст в текущий Document
    if (
        not semantic_snippets[cur_idx].metadata["content_font"]
        or font_size <= semantic_snippets[cur_idx].metadata["content_font"]
    ):
        semantic_snippets[cur_idx].page_content += text_chunk
        semantic_snippets[cur_idx].metadata["content_font"] = max(
            font_size, semantic_snippets[cur_idx].metadata["content_font"]
        )
        continue

    # Если встретился фрагмент с font_size между content_font и heading_font,
    # но не проходит по regex (или другой логике),
    # мы считаем это новым "подзаголовком" (хотя формально может быть не чистым подзаголовком).
    metadata = {
        "heading": text_chunk.strip(),
        "content_font": 0,
        "heading_font": font_size,
    }
    metadata.update(docs[0].metadata)
    semantic_snippets.append(Document(page_content="", metadata=metadata))
    cur_idx += 1

# Пример просмотра результата
print("\nПример полученных фрагментов:")
for idx, docu in enumerate(semantic_snippets[:5]):
    print(f"--- Document #{idx} ---")
    print(f"Заголовок: {docu.metadata['heading']}")
    print(f"Размер шрифта заголовка: {docu.metadata['heading_font']}")
    print(f"Содержимое (первые 100 символов): {docu.page_content[:100]!r}")
    print("-" * 50)


Пример полученных фрагментов:
--- Document #0 ---
Заголовок: Руководство пользователя
Размер шрифта заголовка: 27
Содержимое (первые 100 символов): 'Центральный Пульт 3.16.91\n'
--------------------------------------------------
--- Document #1 ---
Заголовок: Содержание
Размер шрифта заголовка: 14
Содержимое (первые 100 символов): '1\n. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . '
--------------------------------------------------
--- Document #2 ---
Заголовок: 1. Аварии
Размер шрифта заголовка: 20
Содержимое (первые 100 символов): '1.1. Подтвердить аварию\nДля подтверждения аварии требуется выполнить следующие действия:\n1. Нажать к'
--------------------------------------------------
--- Document #3 ---
Заголовок: 2. Классы
Размер шрифта заголовка: 20
Содержимое (первые 100 символов): '2.1. Создание нового класса объектов\nДля создания нового класса нужно выполнить следующие действия:\n'
--------------------------------------------

In [ ]:
len(semantic_snippets)

43

In [ ]:
semantic_snippets[2]

Document(metadata={'heading': '1. Аварии\n', 'content_font': 14, 'heading_font': 20, 'source': 'data/user-guide.pdf'}, page_content='1.1. Подтвердить аварию\nДля подтверждения аварии требуется выполнить следующие действия:\n1. Нажать кнопку  \uf0a2  в панели режимов отображения.\n2. В выпадающем списке вверху окна выбрать "Активные аварии".\n3. Выбрать  аварию,  которую  планируется  отметить  как  подтверждённую,  и  нажать  на  неё\nправой кнопкой мыши.\n4. В открывшемся контекстном меню выбрать "Подтвердить":\nРис. 1. Подтверждение аварии\nПосле этого подтверждённая авария отмечается штрихом, в поле "Подтверждено" будет указано\nимя пользователя и время подтверждения:\nРис. 2. Подтверждённая авария\n1.1.1. REST API\nПодтвердить аварию через REST API можно с помощью запроса Подтвердить аварию.\nДля этого запроса требуется ID аварии.\nПример запроса через curl\nlogin=<...> \npassword=<...> \nsaymon_hostname=<...> \nincident_id=<...> \nurl=https://$saymon_hostname/node/api/incidents/$i